# 数据准备

这个是[RAG技术全栈指南](https://github.com/datawhalechina/all-in-rag)第四章到第五章的学习总结。详细教程请见原文。

数据准备包含如下几个部分：

- 数据读取加载，主要是能读取更多格式的文件，该项工作技术难度不高，只能算工作量😄
- 将文本内容分成大小合适、语义相近的块。切分存在多种不同算法，效果也适合不同场景

## 数据加载

使用Unstructured库加载并解析PDF文件。

In [1]:
from unstructured.partition.auto import partition
import sys
from contextlib import contextmanager

# 定义上下文管理器，过滤指定警告消息
@contextmanager
def filter_stderr_warnings(warning_patterns):
    original_stderr = sys.stderr
    # 自定义 stderr 流，过滤指定警告
    class FilteredStderr:
        def write(self, message):
            # 忽略包含目标警告模式的消息
            if not any(pattern in message for pattern in warning_patterns):
                original_stderr.write(message)
        def flush(self):
            original_stderr.flush()
    
    sys.stderr = FilteredStderr()
    try:
        yield
    finally:
        sys.stderr = original_stderr  # 恢复原始 stderr

# 要过滤的警告消息（完整匹配）
WARNINGS_TO_FILTER = [
    "Could get FontBBox from font descriptor because None cannot be parsed as 4 floats",
    "IProgress not found. Please update jupyter and ipywidgets.",
    "Cannot set gray",
    "short text:",
    "No features in text."
]

with filter_stderr_warnings(WARNINGS_TO_FILTER):

    pdf_file = "data/C2/pdf/rag.pdf"

    elements = partition(
        filename=pdf_file,
        content_type="application/pdf",
    )

    print(f"解析完成: {len(elements)} 个元素, {sum(len(str(e)) for e in elements)} 字符")
    print("=" * 60)

    # 统计元素类型
    from collections import Counter
    types = Counter(e.category for e in elements)
    print(f"元素类型: {dict(types)}")

    print("=" * 60)
    for i, element in enumerate(elements):
        print(f"Element {i} ({element.category}): {element}")
    print("=" * 60)

解析完成: 279 个元素, 7500 字符
元素类型: {'Header': 22, 'Title': 195, 'UncategorizedText': 41, 'NarrativeText': 3, 'Footer': 15, 'ListItem': 3}
Element 0 (Header): 网页
Element 1 (Header): 新闻
Element 2 (Header): 贴吧
Element 3 (Header): 知道
Element 4 (Header): 网盘
Element 5 (Header): 图片
Element 6 (Header): 视频
Element 7 (Header): 地图
Element 8 (Header): 文库
Element 9 (Header): 资讯
Element 10 (Header): 采购
Element 11 (Header): 百科
Element 12 (Header): 百度首页 登录 注册
Element 13 (Title): 检索增强生成
Element 14 (Title): 进⼊词条
Element 15 (Title): 全站搜索
Element 16 (Title): 帮助
Element 17 (Title): 近期有不法分子冒充百度百科官方人员，以删除词条为由威胁并敲诈相关企业。在此严正声明：百度百科是免费编辑平台，绝不存在收费代编服务，请勿上当受骗！ 详情>>
Element 18 (Title): 首页
Element 19 (Title): 秒懂百科
Element 20 (Title): 特色百科
Element 21 (Title): 知识专题
Element 22 (Title): 加入百科
Element 23 (Title): 百科团队
Element 24 (Title): 权威合作
Element 25 (Title): 检索增强生成 播报 ⼤模型前沿技术之⼀ 展开2个同名词条
Element 26 (Title): 锁定
Element 27 (Title): 讨论
Element 28 (UncategorizedText): 1
Element 29 (Title): 上传视频
Element 30 (Title): 一分钟了解检索增强生成 一

调整使用partition_pdf替换当前partition函数，并分别尝试用hi_res和ocr_only进行解析。
需要安装[poppler](https://github.com/oschwartz10612/poppler-windows/releases/)和[Tesseract-OCR](https://github.com/UB-Mannheim/tesseract/wiki)并设置`PATH`环境变量。

In [2]:
from unstructured.partition.pdf import partition_pdf
import sys
from contextlib import contextmanager

# 定义上下文管理器，过滤指定警告消息
@contextmanager
def filter_stderr_warnings(warning_patterns):
    original_stderr = sys.stderr
    # 自定义 stderr 流，过滤指定警告
    class FilteredStderr:
        def write(self, message):
            # 忽略包含目标警告模式的消息
            if not any(pattern in message for pattern in warning_patterns):
                original_stderr.write(message)
        def flush(self):
            original_stderr.flush()
    
    sys.stderr = FilteredStderr()
    try:
        yield
    finally:
        sys.stderr = original_stderr  # 恢复原始 stderr

# 要过滤的警告消息（完整匹配）
WARNINGS_TO_FILTER = [
    "Could get FontBBox from font descriptor because None cannot be parsed as 4 floats",
    "IProgress not found. Please update jupyter and ipywidgets.",
    "Cannot set gray",
    "short text:",
    "No features in text."
]

with filter_stderr_warnings(WARNINGS_TO_FILTER):
    pdf_file = "data/C2/pdf/rag.pdf"
    elements = partition_pdf(
        filename=pdf_file,
        strategy="hi_res",
    )

    print(f"解析完成: {len(elements)} 个元素, {sum(len(str(e)) for e in elements)} 字符")
    print("=" * 60)

    # 统计元素类型
    from collections import Counter
    types = Counter(e.category for e in elements)
    print(f"元素类型: {dict(types)}")

    print("=" * 60)
    for i, element in enumerate(elements):
        print(f"Element {i} ({element.category}): {element}")
    print("=" * 60)

解析完成: 221 个元素, 8265 字符
元素类型: {'Image': 22, 'UncategorizedText': 85, 'Header': 4, 'NarrativeText': 68, 'Table': 4, 'FigureCaption': 4, 'Title': 30, 'ListItem': 4}
Element 0 (Image): 
Element 1 (UncategorizedText): Bh fe
Element 2 (UncategorizedText): Se «6 Be BR 8H Me OE 6B CR OBS
Element 3 (Header): 百度首页 登录 注册
Element 4 (UncategorizedText): 0 0 Bai @ Bil | eme22n
Element 5 (UncategorizedText): x
Element 6 (UncategorizedText): 进⼊词条
Element 7 (UncategorizedText): 全站搜索
Element 8 (UncategorizedText): | mm
Element 9 (NarrativeText): 近期有不法分子冒充百度百科官方人员，以删除词条为由威胁并敲诈相关企业。在此严正声明：百度百科是免费编辑平台，绝不存在收费代编服务，请勿上当受骗！ 详情>>
Element 10 (UncategorizedText): Bn
Element 11 (UncategorizedText): pias
Element 12 (UncategorizedText): HEAR
Element 13 (UncategorizedText): ALIAS a
Element 14 (UncategorizedText): MAB
Element 15 (UncategorizedText): BAAR
Element 16 (UncategorizedText): RAE
Element 17 (Image): 1  检索增强生成 播报  讨论  锁定  上传视频  ⼤模型前沿技术之⼀ 展开2个同名词条  RAG：当下大模型应用落地的必杀技 02:23  01:22  02:23  一分钟了解检索增强生成  01:22  一分钟

In [3]:
from unstructured.partition.pdf import partition_pdf
import sys
from contextlib import contextmanager

# 定义上下文管理器，过滤指定警告消息
@contextmanager
def filter_stderr_warnings(warning_patterns):
    original_stderr = sys.stderr
    # 自定义 stderr 流，过滤指定警告
    class FilteredStderr:
        def write(self, message):
            # 忽略包含目标警告模式的消息
            if not any(pattern in message for pattern in warning_patterns):
                original_stderr.write(message)
        def flush(self):
            original_stderr.flush()
    sys.stderr = FilteredStderr()
    try:
        yield
    finally:
        sys.stderr = original_stderr  # 恢复原始 stderr

# 要过滤的警告消息（完整匹配）
WARNINGS_TO_FILTER = [
    "Could get FontBBox from font descriptor because None cannot be parsed as 4 floats",
    "IProgress not found. Please update jupyter and ipywidgets.",
    "Cannot set gray",
    "short text:",
    "No features in text."
]

with filter_stderr_warnings(WARNINGS_TO_FILTER):
    pdf_file = "data/C2/pdf/rag.pdf"
    elements = partition_pdf(
        filename=pdf_file,
        strategy="ocr_only",
    )

    print(f"解析完成: {len(elements)} 个元素, {sum(len(str(e)) for e in elements)} 字符")
    print("=" * 60)

    # 统计元素类型
    from collections import Counter
    types = Counter(e.category for e in elements)
    print(f"元素类型: {dict(types)}")

    print("=" * 60)
    for i, element in enumerate(elements):
        print(f"Element {i} ({element.category}): {element}")
    print("=" * 60)

解析完成: 137 个元素, 8282 字符
元素类型: {'UncategorizedText': 54, 'Title': 55, 'NarrativeText': 27, 'ListItem': 1}
Element 0 (UncategorizedText): Bh fe Se «6 Be BR 8H Me OE 6B CR OBS ESR SR ith
Element 1 (Title): 0 0 Bai @ Bil | eme22n x SME | mm
Element 2 (NarrativeText): MBA RAATEABEBANEAAR, VRE A BIDHAE RR O, CEL IERAA: BREN RRMA, ONCE CIRS, LYSE! iFIB>> Bn pias HEAR ALIAS a MAB BAAR RAE > 1 REA i oa E AK ) HR Otte Lew
Element 3 (UncategorizedText): ABB AWARAZ—
Element 4 (Title): MoingSh
Element 5 (NarrativeText): fomieas tat OACe >)
Element 6 (NarrativeText): it =
Element 7 (Title): [= = HEhH - SAR RAGFlow RAF RAMA
Element 8 (Title): RAG: = ERIS TEA pares
Element 9 (UncategorizedText): wee) O11) O38
Element 10 (UncategorizedText): Fig) RAG CABAL) ARH ADRHPHNSRALU RMAF SASK, PHASER SSBB, AASPRASRAAE, Lod ie $@901838-E HN ( Retrieval-augmented Generation) , #FRAG, BATA AMEAIARAL— ", aE BY
Element 11 (NarrativeText): MAINE MIRA TS REA SMRRA. RAR, HRVRBEAXALAR SAN, CARM—MEAN aN 2 RARSTOAUBKVES, R

从上面三种方式来看，`auto`方式解析这类非结构化的pdf文件效果看起来更好。`ocr_only`输出有点看不懂。

## 文本分块

文本分块的要求：

- 每一个块里面的内容要语义关联性强
  - 块的语义没有关联会导致计算的向量值出现较大偏差
  - 大量无关的信息填充到大语言模型的上下文可能会导致模型回答质量下降或产生幻觉
- 每一个块的内容长度不能太长
  - 不能超过嵌入模型的长度（如 bge-base-zh-v1.5的上下文窗口为512个token），嵌入模型的作用是将文本块/用户问题转换为一个固定长度的向量（压缩），压缩不可避免有信息损失
  - 大语言模型会将多个相关联的块放到模型上下文中，模型的提示词+上下文+问题的总长度不能超过模型输入长度

从上可以看出：**一个好的文本块应该聚焦于一个明确、单一的主题**（尽可能简短）。

## 分块策略

如何分块存在如下几种方式：

- 固定大小分块
  - 最简单的分块方式，首先按段落分割，字数不够多个段落来凑；
  - 单个段落超出长度还是保留（ `chunk_size` 其实就是建议）
  - 通过重叠机制（`chunk_overlap`）保持上下文连续性
- 递归字符分块
  - 通过一系列的更细粒度的分隔符先将文本分割为片段
  - 递归以找到满足大小的多个块
- 语义分块
- 基于文档分块
- 基于文档元素的智能分块

In [3]:
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader

loader = TextLoader("data/C2/txt/蜂医.txt")
docs = loader.load()

text_spliter = CharacterTextSplitter(chunk_size=200, chunk_overlap=10)
chunks = text_spliter.split_documents(docs)

print(f"文本分块完成: {len(chunks)} 个块")
print("=" * 40)
for i, chunk in enumerate(chunks):
    print(f"Chunk {i}: {chunk.page_content}")
    print("=" * 40)

Created a chunk of size 201, which is longer than the specified 200
Created a chunk of size 277, which is longer than the specified 200
Created a chunk of size 296, which is longer than the specified 200


文本分块完成: 14 个块
Chunk 0: # 蜂医

游戏《三角洲行动》中的支援型干员

蜂医是2024年琳琅天上发行的《三角洲行动》中的支援型干员之一，在早期版本是唯一一个支援型干员。
Chunk 1: 蜂医在游戏中能够使用战术装备“激素枪”：远程治疗队友或'自我治疗'，还可以使用兵种道具“烟幕无人机”：释放长烟幕，和“蜂巢科技烟雾弹”：产生一团白色烟雾（使用激素枪对烟雾射击换变成绿色烟雾，可起到治疗作用），干员特长为“高效救援”：救援倒地队友时速度更快，在全面战场模式中约1.4秒就能救起队友，且被救起的队友能恢复更多生命值。在烽火地带中，还能够移除队友血量上限减少的负面效果。 \[1-2]****
Chunk 2: * 中文名

  罗伊•斯米

* 外文名

  Roy smee \[2]**

* 别    名

  罗伊、蜂医

* 性    别

  男

- 登场作品

  [三角洲行动](/item/%E4%B8%89%E8%A7%92%E6%B4%B2%E8%A1%8C%E5%8A%A8/63251542?fromModule=lemma_inlink)

- 生    日
Chunk 3: - 生    日

  2008年2月23日 \[3]**

- 身    高

  176 cm \[3]**

- 体    重

  75 kg \[3]**

## 目录

1. 1[角色设定](#1)
2. 2[角色定位](#2)
3. 3[技能](#3)
Chunk 4: 1) ▪[战术装备 - 激素枪](#3-1)
2) ▪[战术道具 - 烟幕无人机](#3-2)
3) ▪[战术道具 - 蜂巢科技烟雾弹](#3-3)

1. ▪[干员特长 - 高效救援](#3-4)

## 角色设定

三角洲行动：医疗角色蜂医！让你不再为血量担忧

蜂医是游戏中的一名战地医生，拥有丰富的参军履历。

蜂医在干员档案中标明他有一个妻子和女儿。

## 角色定位
Chunk 5: ## 角色定位

-蜂医在团队中主要承担支援和治疗的职责，是团队生存能力的重要保障。他能在激烈的战斗中为队友提供及时的治疗，确保队友保持良好的战斗状态，还能利用烟幕技能为团队创造有利的战术环境，如掩护队友转移、推进或突袭等。

## 技能

### 战术装备 - 激素枪

-可发射多目

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader

loader = TextLoader("data/C2/txt/蜂医.txt")
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", "。", "，", " ", ""],  # 分隔符优先级
    chunk_size=200,
    chunk_overlap=10,
)

chunks = text_splitter.split_documents(docs)
print(f"文本分块完成: {len(chunks)} 个块")
print("=" * 40)

for i, chunk in enumerate(chunks):
    print(f"Chunk {i}: {chunk.page_content}")
    print("=" * 40)

文本分块完成: 22 个块
Chunk 0: # 蜂医

游戏《三角洲行动》中的支援型干员

蜂医是2024年琳琅天上发行的《三角洲行动》中的支援型干员之一，在早期版本是唯一一个支援型干员。
Chunk 1: 蜂医在游戏中能够使用战术装备“激素枪”：远程治疗队友或'自我治疗'，还可以使用兵种道具“烟幕无人机”：释放长烟幕，和“蜂巢科技烟雾弹”：产生一团白色烟雾（使用激素枪对烟雾射击换变成绿色烟雾，可起到治疗作用），干员特长为“高效救援”：救援倒地队友时速度更快，在全面战场模式中约1.4秒就能救起队友，且被救起的队友能恢复更多生命值。在烽火地带中，还能够移除队友血量上限减少的负面效果
Chunk 2: 。 \[1-2]****
Chunk 3: * 中文名

  罗伊•斯米

* 外文名

  Roy smee \[2]**

* 别    名

  罗伊、蜂医

* 性    别

  男

- 登场作品

  [三角洲行动](/item/%E4%B8%89%E8%A7%92%E6%B4%B2%E8%A1%8C%E5%8A%A8/63251542?fromModule=lemma_inlink)

- 生    日
Chunk 4: - 生    日

  2008年2月23日 \[3]**

- 身    高

  176 cm \[3]**

- 体    重

  75 kg \[3]**

## 目录

1. 1[角色设定](#1)
2. 2[角色定位](#2)
3. 3[技能](#3)
Chunk 5: 1) ▪[战术装备 - 激素枪](#3-1)
2) ▪[战术道具 - 烟幕无人机](#3-2)
3) ▪[战术道具 - 蜂巢科技烟雾弹](#3-3)

1. ▪[干员特长 - 高效救援](#3-4)

## 角色设定

三角洲行动：医疗角色蜂医！让你不再为血量担忧

蜂医是游戏中的一名战地医生，拥有丰富的参军履历。

蜂医在干员档案中标明他有一个妻子和女儿。

## 角色定位
Chunk 6: ## 角色定位

-蜂医在团队中主要承担支援和治疗的职责，是团队生存能力的重要保障。他能在激烈的战斗中为队友提供及时的治疗，确保队友保持良好的战斗状态，还能利用烟幕技能为团队创造有利的战术环境，如掩护队友转移、推进或突袭等。

## 技能

### 战术装备 - 激

In [9]:
import os
from langchain_experimental.text_splitter import SemanticChunker
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import TextLoader

embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-small-zh-v1.5",
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)

# 初始化 SemanticChunker
text_splitter = SemanticChunker(
    embeddings,
    breakpoint_threshold_type="percentile" # 断点识别方法
)

loader = TextLoader("data/C2/txt/蜂医.txt")
docs = loader.load()

chunks = text_splitter.split_documents(docs)
print(f"文本分块完成: {len(chunks)} 个块")
print("=" * 40)
for i, chunk in enumerate(chunks):
    print(f"Chunk {i}: {chunk.page_content}")
    print("=" * 40)


文本分块完成: 2 个块
Chunk 0: # 蜂医

游戏《三角洲行动》中的支援型干员

蜂医是2024年琳琅天上发行的《三角洲行动》中的支援型干员之一，在早期版本是唯一一个支援型干员。

蜂医在游戏中能够使用战术装备“激素枪”：远程治疗队友或'自我治疗'，还可以使用兵种道具“烟幕无人机”：释放长烟幕，和“蜂巢科技烟雾弹”：产生一团白色烟雾（使用激素枪对烟雾射击换变成绿色烟雾，可起到治疗作用），干员特长为“高效救援”：救援倒地队友时速度更快，在全面战场模式中约1.4秒就能救起队友，且被救起的队友能恢复更多生命值。在烽火地带中，还能够移除队友血量上限减少的负面效果。 \[1-2]****

* 中文名

  罗伊•斯米

* 外文名

  Roy smee \[2]**

* 别    名

  罗伊、蜂医

* 性    别

  男

- 登场作品

  [三角洲行动](/item/%E4%B8%89%E8%A7%92%E6%B4%B2%E8%A1%8C%E5%8A%A8/63251542?fromModule=lemma_inlink)

- 生    日

  2008年2月23日 \[3]**

- 身    高

  176 cm \[3]**

- 体    重

  75 kg \[3]**

## 目录

1. 1[角色设定](#1)
2.
Chunk 1: 2[角色定位](#2)
3. 3[技能](#3)

1) ▪[战术装备 - 激素枪](#3-1)
2) ▪[战术道具 - 烟幕无人机](#3-2)
3) ▪[战术道具 - 蜂巢科技烟雾弹](#3-3)

1. ▪[干员特长 - 高效救援](#3-4)

## 角色设定

三角洲行动：医疗角色蜂医！让你不再为血量担忧

蜂医是游戏中的一名战地医生，拥有丰富的参军履历。

蜂医在干员档案中标明他有一个妻子和女儿。

## 角色定位

-蜂医在团队中主要承担支援和治疗的职责，是团队生存能力的重要保障。他能在激烈的战斗中为队友提供及时的治疗，确保队友保持良好的战斗状态，还能利用烟幕技能为团队创造有利的战术环境，如掩护队友转移、推进或突袭等。

## 技能

### 战术装备 - 激素枪

-可发射多目标追踪子弹，能同时锁定并缓慢治疗多名友方干员，治疗期间还能抑制队友的疼痛感等异常状态。

-开镜时，仅

In [14]:

# 初始化 SemanticChunker
text_splitter = SemanticChunker(
    embeddings,
    breakpoint_threshold_type="standard_deviation" # 断点识别方法
)

loader = TextLoader("data/C2/txt/蜂医.txt")
docs = loader.load()

chunks = text_splitter.split_documents(docs)
print(f"文本分块完成: {len(chunks)} 个块")

文本分块完成: 1 个块


In [19]:
from langchain.text_splitter import MarkdownHeaderTextSplitter
from unstructured.partition.auto import partition

md_path = "data/C2/txt/蜂医.md"

elements = partition(filename=md_path)
# 打印解析结果
print(f"解析完成: {len(elements)} 个元素, {sum(len(str(e)) for e in elements)} 字符")

text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", "。", "，", " ", ""],  # 分隔符优先级
    chunk_size=200,
    chunk_overlap=10,
)

chunks = []
chunk = ""
has_content = False
for i, element in enumerate(elements):
    if element.category == "Title":
        if has_content:
            chunks.append(text_splitter.split_text(chunk))
            chunk = ""
            has_content = False
        chunk += element.text + "\n"
    elif element.category == "Image":
        continue
    else:
        chunk += element.text + "\n"
        has_content = True

print("=" * 40)
for i, chunk in enumerate(chunks):
    print(f"Chunk {i}: {chunk}")
    print("=" * 40)




libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
short text: "Roy smee [2]**". Defaulting to English.
short text: "176 cm [3]**". Defaulting to English.
short text: "75 kg [3]**". Defaulting to English.
short text: "1". Defaulting to English.


解析完成: 53 个元素, 1246 字符
Chunk 0: ['蜂医\n游戏《三角洲行动》中的支援型干员\n蜂医是2024年琳琅天上发行的《三角洲行动》中的支援型干员之一，在早期版本是唯一一个支援型干员。', "蜂医在游戏中能够使用战术装备“激素枪”：远程治疗队友或'自我治疗'，还可以使用兵种道具“烟幕无人机”：释放长烟幕，和“蜂巢科技烟雾弹”：产生一团白色烟雾（使用激素枪对烟雾射击换变成绿色烟雾，可起到治疗作用），干员特长为“高效救援”：救援倒地队友时速度更快，在全面战场模式中约1.4秒就能救起队友，且被救起的队友能恢复更多生命值。在烽火地带中，还能够移除队友血量上限减少的负面效果", '。 [1-2]****', '中文名\n罗伊•斯米\n外文名\nRoy smee [2]**\n别 名\n罗伊、蜂医\n性 别\n登场作品\n三角洲行动\n生 日\n2008年2月23日 [3]**\n身 高\n176 cm [3]**\n体 重\n75 kg [3]**']
Chunk 1: ['目录\n1角色设定\n2角色定位\n3技能\n1) ▪战术装备 - 激素枪 2) ▪战术道具 - 烟幕无人机 3) ▪战术道具 - 蜂巢科技烟雾弹\n▪干员特长 - 高效救援']
Chunk 2: ['角色设定\n三角洲行动：医疗角色蜂医！让你不再为血量担忧\n蜂医是游戏中的一名战地医生，拥有丰富的参军履历。\n蜂医在干员档案中标明他有一个妻子和女儿。']
Chunk 3: ['角色定位\n蜂医在团队中主要承担支援和治疗的职责，是团队生存能力的重要保障。他能在激烈的战斗中为队友提供及时的治疗，确保队友保持良好的战斗状态，还能利用烟幕技能为团队创造有利的战术环境，如掩护队友转移、推进或突袭等。']
Chunk 4: ['技能\n战术装备 - 激素枪\n可发射多目标追踪子弹，能同时锁定并缓慢治疗多名友方干员，治疗期间还能抑制队友的疼痛感等异常状态。\n开镜时，仅会锁定一个目标。\n全面战场中，开镜瞄准敌人可以扣除敌人20滴血\n长按按键可以对自己进行治疗，在全面战场中，可以恢复自身血量。烽火地带模式中，可以消除自身如骨折的负面效果。']
Chunk 5: ['战术道具 - 烟幕无人机\n能控制手势感应无人机向前飞行并喷射烟幕，形成一条长15米的烟雾长廊\n无人机飞行时，长按开火键引导无人

从上可以看出，其实结构化文档更有利用分块，毕竟人工先分了，比机器要好很多。
一个很奇怪的问题是：为什么语义分块的效果看起来比较糟糕！

后续可以优先考虑基于**文档结构的分块 + 递归字符分块**，看起来效果更佳。